--- Day 15: Beacon Exclusion Zone ---

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:
```
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
```
So, consider the sensor at 2,18; the closest beacon to it is at -2,15. For the sensor at 9,16, the closest beacon to it is at 10,16.

Drawing sensors as S and beacons as B, the above arrangement of sensors and beacons looks like this:
```
               1    1    2    2
     0    5    0    5    0    5
 0 ....S.......................
 1 ......................S.....
 2 ...............S............
 3 ................SB..........
 4 ............................
 5 ............................
 6 ............................
 7 ..........S.......S.........
 8 ............................
 9 ............................
10 ....B.......................
11 ..S.........................
12 ............................
13 ............................
14 ..............S.......S.....
15 B...........................
16 ...........SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```
This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:
```
               1    1    2    2
     0    5    0    5    0    5
-2 ..........#.................
-1 .........###................
 0 ....S...#####...............
 1 .......#######........S.....
 2 ......#########S............
 3 .....###########SB..........
 4 ....#############...........
 5 ...###############..........
 6 ..#################.........
 7 .#########S#######S#........
 8 ..#################.........
 9 ...###############..........
10 ....B############...........
11 ..S..###########............
12 ......#########.............
13 .......#######..............
14 ........#####.S.......S.....
15 B........###................
16 ..........#SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```
This sensor's closest beacon is at 2,10, and so you know there are no beacons that close or closer (in any positions marked #).

None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:
```
                 1    1    2    2
       0    5    0    5    0    5
 9 ...#########################...
10 ..####B######################..
11 .###S#############.###########.
```
In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?


In [35]:
// Data Types and Functions
using System.Text.RegularExpressions;

public record struct Position {
    public int X;
    public int Y;

    public static Position operator+(Position position, Position other) {
        return new Position {
            X = position.X + other.X,
            Y = position.Y + other.Y,
        };
    }
    
    public static Position operator-(Position position, Position other) {
        return new Position {
            X = position.X - other.X,
            Y = position.Y - other.Y,
        };
    }
}

public record struct SensorReading {
    public Position Sensor;
    public Position NearestBeacon;

    public int dist => Math.Abs(Sensor.X - NearestBeacon.X) + Math.Abs(Sensor.Y - NearestBeacon.Y);

    // MinX may be higher than MaxX if the row is further away than the closest beacon
    // This is intentional as it helps us in part 2 - we can adjust the covered range each row
    // instead of recalculating this each time
    public Bounds CoveredBoundsInRow(int y) {
        var radius = dist - Math.Abs(Sensor.Y - y);
        return new Bounds {
            MinX = Sensor.X - radius,
            MaxX = Sensor.X + radius,
            MinY = y,
            MaxY = y,
        };
    }
}

var SensorReadingRegex = new Regex(@"Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)");
SensorReading ParseLine(string line) {
    var matchGroups = SensorReadingRegex.Match(line).Groups;
    return new SensorReading {
        Sensor = new Position {
            X = int.Parse(matchGroups[1].Value),
            Y = int.Parse(matchGroups[2].Value),
        },
        NearestBeacon = new Position {
            X = int.Parse(matchGroups[3].Value),
            Y = int.Parse(matchGroups[4].Value),
        },
    };
}

// I tried using record stucts as usual, but it seems in part 2 the overhead of recreating bounds
// every row rather than adjusting existing ones was a significant part of the slowdown
public class Bounds {
    public int MinX { get; set; }
    public int MaxX { get; set; }
    public int MinY { get; set; }
    public int MaxY { get; set; }
}

public class RowAndBounds {
    public int SensorY { get; set; }
    public Bounds Bounds { get; set; }
}

Position? FindBeacon(IEnumerable<SensorReading> readings, Bounds bounds) {
    var rowBounds = readings.Select(reading => new RowAndBounds {
            SensorY = reading.Sensor.Y,
            Bounds = reading.CoveredBoundsInRow(bounds.MinY)
        })
        .ToList();

    for (var y = bounds.MinY; y <= bounds.MaxY; y++) {
        for (var x = bounds.MinX; x <= bounds.MaxX; x++) {
            var covered = false;
            
            // Don't bother checking every cell, just skip to the end of each relevant sensor's
            // covering range within this row when we hit it. There aren't many sensors so this
            // doesn't scale too badly
            foreach(var rowbound in rowBounds) {
                if (x >= rowbound.Bounds.MinX && x <= rowbound.Bounds.MaxX) {
                    covered = true;
                    x = rowbound.Bounds.MaxX;
                }
            }
            if (!covered) {
                return new Position { X = x, Y = y };
            }
        }

        // Adjust bounds rather than recalculating them each row
        rowBounds.ForEach(rowBound => {
            if (y >= rowBound.SensorY) {
                rowBound.Bounds.MinX += 1;
                rowBound.Bounds.MaxX -= 1;
            } else {
                rowBound.Bounds.MinX -= 1;
                rowBound.Bounds.MaxX += 1;
            } 
        });
    }
    return null;
}

In [36]:
var input = System.IO.File.ReadAllLines("input.txt");

/*
var row = 10;
var bounds = new Bounds {
    MinX = 0,
    MinY = 0,
    MaxX = 20,
    MaxY = 20,
};
*/

var row = 2000000;
var bounds = new Bounds {
    MinX = 0,
    MinY = 0,
    MaxX = 4000000,
    MaxY = 4000000,
};

input.Take(10)

index,value
0,"Sensor at x=251234, y=759482: closest beacon is at x=-282270, y=572396"
1,"Sensor at x=2866161, y=3374117: closest beacon is at x=2729330, y=3697325"
2,"Sensor at x=3999996, y=3520742: closest beacon is at x=3980421, y=3524442"
3,"Sensor at x=3988282, y=3516584: closest beacon is at x=3980421, y=3524442"
4,"Sensor at x=3005586, y=3018139: closest beacon is at x=2727127, y=2959718"
5,"Sensor at x=3413653, y=3519082: closest beacon is at x=3980421, y=3524442"
6,"Sensor at x=2900403, y=187208: closest beacon is at x=2732772, y=2000000"
7,"Sensor at x=1112429, y=3561166: closest beacon is at x=2729330, y=3697325"
8,"Sensor at x=3789925, y=3283328: closest beacon is at x=3980421, y=3524442"
9,"Sensor at x=3991533, y=3529053: closest beacon is at x=3980421, y=3524442"


In [37]:
// Part 1
var readings = input.Select(ParseLine);
var knownBeacons = readings.Select(reading => reading.NearestBeacon)
    .ToHashSet();
readings.Select(reading => reading.CoveredBoundsInRow(row))
    .Where(bounds => bounds.MinX <= bounds.MaxX)
    .SelectMany(bounds => Enumerable.Range(bounds.MinX, Math.Max(0, bounds.MaxX - bounds.MinX + 1)))
    .Select(x => new Position { X = x, Y = row })
    .ToHashSet()
    .Except(knownBeacons)
    .Count()

5688618

Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have x and y coordinates each no lower than 0 and no larger than 4000000.

To isolate the distress beacon's signal, you need to determine its tuning frequency, which can be found by multiplying its x coordinate by 4000000 and then adding its y coordinate.

In the example above, the search space is smaller: instead, the x and y coordinates can each be at most 20. With this reduced search area, there is only a single position that could have a beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

Find the only possible position for the distress beacon. What is its tuning frequency?

In [38]:
// Part 2
var readings = input.Select(ParseLine);
Position beacon = (Position) FindBeacon(readings, bounds);
(long) beacon.X * 4000000 + (long) beacon.Y

12625383204261